In [33]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [37]:
# 加载数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
# 数据加载机制
# 通过trainloader 完成当前批次输入量为4次，工作线程4次
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
# 数据加载机制
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [35]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        #
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# 训练模型
for epoch in range(2):  # 多次循环遍历数据集
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000个小批量打印一次损失值
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.192
[1,  4000] loss: 1.854
[1,  6000] loss: 1.683
[1,  8000] loss: 1.577
[1, 10000] loss: 1.504
[1, 12000] loss: 1.443
[2,  2000] loss: 1.399
[2,  4000] loss: 1.357
[2,  6000] loss: 1.319
[2,  8000] loss: 1.305
[2, 10000] loss: 1.279
[2, 12000] loss: 1.255
Finished Training


In [36]:
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        """
        这个操作是用来获取模型输出中最大值所在的索引，即预测的类别标签。
        具体来说，假设`outputs`是一个形状为`(batch_size, num_classes)`的张量，表示模型对于输入样本的输出结果。其中，`batch_size`是批次大小，`num_classes`是分类任务的类别数量。则`outputs.data`是一个形状为`(batch_size, num_classes)`的张量，表示模型输出的原始数据，每一行对应一个样本输出的概率分布。
        `torch.max()`函数会返回张量中最大值和最大值所在的索引。因此，`_, predicted = torch.max(outputs.data, 1)`这行代码会将模型输出中最大值所在的索引存储在`predicted`变量中，忽略最大值本身。
        最后，`predicted`变量中存储的就是模型对于输入样本的预测类别标签。例如，如果`predicted`的第$i$个元素为$j$，则表示模型预测第$i$个样本属于第$j$个类别。
        """
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 56 %


1.假设$x$为$m$ 维随机变量，其均值为$\mu$，协方差矩阵为$\Sigma$。

考虑由$m$维随机变量$x$到$m$维随机变量$y$的线性变换
$$y _ { i } = \alpha _ { i } ^ { T } x = \sum _ { k = 1 } ^ { m } \alpha _ { k i } x _ { k } , \quad i = 1,2 , \cdots , m$$

其中$\alpha _ { i } ^ { T } = ( \alpha _ { 1 i } , \alpha _ { 2 i } , \cdots , \alpha _ { m i } )$。